In [1]:
import csv
import pandas as pd

from docx import Document

## Code

In [28]:
def docx_to_csv(doc_path, csv_path):
    """
    Read tables from docx document to csv without preprocessing
    
    Parameters:
        doc_path:  path to docx document (read)
        csv_path:  path to csv document (save)
    """
    doc = Document(doc_path)
    
    with open(csv_path, 'w') as f:
        writer = csv.writer(f)
        for table in doc.tables:
            for row in table.rows:
                # replacing '\n' makes 'PRE-CONDITION' fit on single line
                # end evoids error while spliting row
                writer.writerow(cell.text.replace('\n', '|n').strip(' |n')
                                for cell in row.cells)
            
            writer.writerow(['TABLE_END'])
            
    print('Done!') 

In [33]:
class TableProcessor:
    def __init__(self, raw_df, countries_df=False, clean_df=False, pre_df=False, country=False):
        self.table_name_col = 'CONNECTION TYPE'
        self.country_col = 'COUNTRY'
        self.pre_col = 'PRE-CONDITION'
        self.table_end = 'TABLE_END'
        
        self.rdf = raw_df
        self.cdf = clean_df if clean_df else pd.DataFrame()
        self.pre_df = pre_df if pre_df else pd.DataFrame()
        self.pre_id = False
        self.countries_df = (countries_df 
                             if isinstance(countries_df, pd.DataFrame) 
                             else pd.DataFrame({'id':[0], self.country_col:[None]}))
        self.country=country
        
        # TODO
        # fix country setup when given 'country' argument
#         self.set_country_id() 
        
        
    def process_tables(self):
        rows = csv.reader(self.rdf[0])
        
        for row in rows:
            
            if self.is_table_end(row):
                self.reset(row)
            
            elif self.is_country(row):
                self.set_country(row)
            
            elif self.is_table_name(row):
                self.set_table_name(row)
                
            elif self.is_table_header(row):
                self.set_table_header(row)
                
            elif self.is_precondition(row):
                self.set_precondition(row)
                
            else:
                self.set_row(row)
                
        self.handle_outcome()
        return self.cdf, self.countries_df, self.pre_df
    
    # RESET
    
    def is_table_end(self, row):
        return row[0] == self.table_end
    
    def reset(self, row):
        self.pre_id = False
    
    # COUNTRY
    def is_country(self, row):
        return row[0].startswith('COUNTRY')
    
    def set_country(self, row):
        if not self.country:
            self.country = row[0].split(':')[1].strip()
            self.set_country_id()
            
    def set_country_id(self):        
        if self.country in self.countries_df[self.country_col].values:
            mask = self.countries_df[self.country_col] == self.country
            self.country_id = countries_df[mask].id.values[0]
        else:
            self.country_id = self.countries_df['id'].max() + 1
            new_country = {'id': self.country_id, 
                           self.country_col: self.country}
            self.countries_df = self.countries_df.append(new_country, 
                                                         ignore_index=True)
    # TABLE NAME
    def is_table_name(self, row):
        return len(row) > 1 and len(set(row)) == 1 and all(map(str.isupper, row))
    
    def set_table_name(self, row):
        self.table_name = row[0]
    
    # TABLE HEADER
    def is_table_header(self, row):
        return all(map(str.isupper, row))
    
    def set_table_header(self, row):
        self.columns = {idx: column for idx, column in enumerate(row)}
        for column in self.columns.values():
            if column not in self.cdf.columns:
                self.cdf[column] = np.nan
                
    # PRE-CONDITION
    def is_precondition(self, row):
        return row[0].startswith('PRE-CONDITION')        
    
    def set_precondition(self, row):
        self.pre_id = 1
        new_precondition = {
            'id': self.pre_id, 
            self.pre_col: row[0],
        }
        if len(self.pre_df):
            self.pre_id = self.pre_df['id'].max() + 1
            new_precondition['id'] = self.pre_id
            self.pre_df = self.pre_df.append(new_precondition, 
                                             ignore_index=True)
        else:
            self.pre_df = self.pre_df.append(new_precondition, 
                                             ignore_index=True)
    
    # TABLE ROW OF DATA
    def set_row(self, row):
        self.check_country()
        
        dict_row = {self.columns[col_id]: val for col_id, val in enumerate(row)}
        dict_row[self.table_name_col] = self.table_name
        dict_row[self.country_col] = self.country_id
        if self.pre_id:
            dict_row[self.pre_col] = self.pre_id
        
        self.cdf = self.cdf.append(dict_row, ignore_index=True)
        
    # OUTCOME, SCENARIO
    def handle_outcome(self):
        self.merge_scenarios()
        self.clean_outcome_value()
        
    def merge_scenarios(self):
        id_vars = [c for c in self.cdf.columns if not c.endswith('SCENARIO')]
        self.cdf = self.cdf.melt(id_vars=id_vars)
        self.cdf.rename(columns={'value': 'OUTCOME', 
                                 'variable': 'SCENARIO'}, inplace=True)
    
    def clean_outcome_value(self):
        not_outcome = self.cdf.columns.drop('OUTCOME').tolist()
        
        self.cdf = self.cdf.set_index(not_outcome)['OUTCOME']\
            .str.split('\|n', expand=True)\
            .stack().reset_index()\
            .rename(columns={0:'OUTCOME'})\
            .loc[:, self.cdf.columns]
    
    # OTHER
    def check_country(self):
        try:
            self.country_id
        except AttributeError as e:
            e.args = (('Country not found! '
                      'Please check input document,'
                      'or pass country as an argument'), *e.args)
            raise 
    

## Test one file

### Swiss (regular tables)

In [24]:
doc_swiss = "../res/swiss.docx"
csv_swiss = "../res/swiss.csv"

docx_to_csv(doc_swiss, csv_swiss)

Done!


In [5]:
swiss_raw_df = pd.read_csv(csv_swiss, header=None, sep='\n')
swiss_raw_df

,0
0,COUNTRY: SWITZERLAND
1,"PROSPECTS,PROSPECTS,PROSPECTS,PROSPECTS,PROSPE..."
2,"GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,OF..."
3,"Social Contact,Social interactions,Retail,Pro-..."
4,"Social Contact,Social interactions,Retail,Upon..."
...,...
120,"Marketing of Specific Services,Corporate Finan..."
121,"Marketing of Specific Services,Wealth services..."
122,"Marketing of Specific Services,Wealth services..."
123,"Marketing of Specific Services,Wealth services..."


In [6]:
tp = TableProcessor(swiss_raw_df)
swiss_df, countries_df, precondition_df = tp.process_tables()

In [7]:
countries_df

,id,COUNTRY
0,0,None
1,1,SWITZERLAND


In [8]:
precondition_df

""


In [9]:
swiss_df.head()

,GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,CONNECTION TYPE,COUNTRY,SCENARIO,OUTCOME
0,Social Contact,Social interactions,Retail,Pro-actively,PROSPECTS,1.0,OFFSHORE SCENARIO,YES
1,Social Contact,Social interactions,Retail,Upon RS,PROSPECTS,1.0,OFFSHORE SCENARIO,YES
2,Social Contact,Social interactions,Professionals,Pro-actively,PROSPECTS,1.0,OFFSHORE SCENARIO,YES
3,Social Contact,Social interactions,Professionals,Upon RS,PROSPECTS,1.0,OFFSHORE SCENARIO,YES
4,Social Contact,Provide contact details,Retail,Pro-actively,PROSPECTS,1.0,OFFSHORE SCENARIO,YES


### Germany (tables with precondition and multiple YES/NO in cell)

In [34]:
doc_ger = "../res/germany.docx"
csv_ger = "../res/germany.csv"

docx_to_csv(doc_ger, csv_ger)

Done!


In [35]:
ger_raw_df = pd.read_csv(csv_ger, header=None, sep='\n')
ger_raw_df

,0
0,COUNTRY: GERMANY
1,TABLE_END
2,PRE-CONDITION FOR RELYING ON PREPARATORY ADMIN...
3,"PROSPECTS,PROSPECTS,PROSPECTS,PROSPECTS,PROSPE..."
4,"GROUP,ACTIVITY,SUBACTIVITY,CLIENT TYPE,INTERAC..."
...,...
69,"Preparatory administrative activities,""Onboard..."
70,"Preparatory administrative activities,""Onboard..."
71,"Preparatory administrative activities,""Onboard..."
72,"Preparatory administrative activities,""Onboard..."


In [36]:
tp = TableProcessor(ger_raw_df, countries_df)
ger_df, countries_df, precondition_df = tp.process_tables()

In [37]:
countries_df

,id,COUNTRY
0,0,None
1,1,SWITZERLAND
2,2,GERMANY


In [38]:
precondition_df

,PRE-CONDITION,id
0,PRE-CONDITION FOR RELYING ON PREPARATORY ADMIN...,1.0
1,PRE-CONDITION FOR RELYING ON PREPARATORY ADMIN...,2.0


In [39]:
precondition_df.loc[0, 'PRE-CONDITION'].split('|n')

['PRE-CONDITION FOR RELYING ON PREPARATORY ADMINISTRATIVE ACTIVITIES',
 'Any activity below subject to �YES� shall only be conducted:',
 '1) if the condition(s) for the marketing of the relevant service(s) is (are) met OR',
 '2) when the activity is necessary to update the documents and information relating to existing clients']

In [40]:
ger_df

,GROUP,ACTIVITY,SUBACTIVITY,CLIENT TYPE,INTERACTION MODE,CONNECTION TYPE,COUNTRY,PRE-CONDITION,SCENARIO,OUTCOME
0,Preparatory administrative activities,Provide forms and contractual documents,Account opening contracts including related forms,Retail,Pro-actively,PROSPECTS,2.0,1.0,OFFSHORE SCENARIO,NO
1,Preparatory administrative activities,Provide forms and contractual documents,Account opening contracts including related forms,Retail,Upon RS,PROSPECTS,2.0,1.0,OFFSHORE SCENARIO,YES with a fiduciary company and subject to an...
2,Preparatory administrative activities,Provide forms and contractual documents,Account opening contracts including related forms,Retail,Upon RS,PROSPECTS,2.0,1.0,OFFSHORE SCENARIO,NO without a fiduciary company (exceptional ca...
3,Preparatory administrative activities,Provide forms and contractual documents,Account opening contracts including related forms,Professionals,Pro-actively,PROSPECTS,2.0,1.0,OFFSHORE SCENARIO,NO
4,Preparatory administrative activities,Provide forms and contractual documents,Account opening contracts including related forms,Professionals,Upon RS,PROSPECTS,2.0,1.0,OFFSHORE SCENARIO,YES with a fiduciary company and subject to an...
...,...,...,...,...,...,...,...,...,...,...
127,Preparatory administrative activities,"Onboarding formality 1-3, including subsequent...","Onboarding formality 1-3, including subsequent...",Professionals,Upon RS,EXISTING CLIENTS,2.0,2.0,ONSHORE SCENARIO,NO
128,Preparatory administrative activities,"Onboarding formality 4, including subsequent u...","Onboarding formality 4, including subsequent u...",Retail,Pro-actively,EXISTING CLIENTS,2.0,2.0,ONSHORE SCENARIO,NO
129,Preparatory administrative activities,"Onboarding formality 4, including subsequent u...","Onboarding formality 4, including subsequent u...",Retail,Upon RS,EXISTING CLIENTS,2.0,2.0,ONSHORE SCENARIO,NO
130,Preparatory administrative activities,"Onboarding formality 4, including subsequent u...","Onboarding formality 4, including subsequent u...",Professionals,Pro-actively,EXISTING CLIENTS,2.0,2.0,ONSHORE SCENARIO,NO


### test-swiss (whole document)

In [41]:
doc_all = "../res/test-swiss.docx"
csv_all = "../res/test-swiss.csv"

docx_to_csv(doc_all, csv_all)

Done!


In [42]:
all_raw_df = pd.read_csv(csv_all, header=None, sep='\n')
all_raw_df

,0
0,COUNTRY: SWITZERLAND
1,TABLE_END
2,"PROSPECTS,PROSPECTS,PROSPECTS,PROSPECTS,PROSPE..."
3,"GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,OF..."
4,"Social Contact,Social interactions,Retail,Pro-..."
...,...
312,"Follow-up administrative tasks,""Provide accoun..."
313,"Follow-up administrative tasks,""Provide accoun..."
314,"Follow-up administrative tasks,""Provide accoun..."
315,"Follow-up administrative tasks,""Provide accoun..."


In [43]:
tp = TableProcessor(all_raw_df, countries_df)
all_df, countries_df, precondition_df = tp.process_tables()

In [50]:
all_df

,GROUP,ACTIVITY,CLIENT TYPE,INTERACTION MODE,CONNECTION TYPE,COUNTRY,SUBACTIVITY,PRE-CONDITION,CONTRACTUAL SETUP,SCENARIO,OUTCOME
0,Social Contact,Social interactions,Retail,Pro-actively,PROSPECTS,1.0,NaN,NaN,NaN,OFFSHORE SCENARIO,YES
1,Social Contact,Social interactions,Retail,Upon RS,PROSPECTS,1.0,NaN,NaN,NaN,OFFSHORE SCENARIO,YES
2,Social Contact,Social interactions,Professionals,Pro-actively,PROSPECTS,1.0,NaN,NaN,NaN,OFFSHORE SCENARIO,YES
3,Social Contact,Social interactions,Professionals,Upon RS,PROSPECTS,1.0,NaN,NaN,NaN,OFFSHORE SCENARIO,YES
4,Social Contact,Provide contact details,Retail,Pro-actively,PROSPECTS,1.0,NaN,NaN,NaN,OFFSHORE SCENARIO,YES
...,...,...,...,...,...,...,...,...,...,...,...
559,Follow-up administrative tasks,Corporate actions|nAccount closure,Professionals,Upon RS,EXISTING CLIENTS,1.0,NaN,NaN,NaN,ONSHORE SCENARIO,YES
560,Follow-up administrative tasks,"Provide account statements, investment reports...",Retail,Pro-actively,EXISTING CLIENTS,1.0,NaN,NaN,NaN,ONSHORE SCENARIO,NO
561,Follow-up administrative tasks,"Provide account statements, investment reports...",Retail,Upon RS,EXISTING CLIENTS,1.0,NaN,NaN,NaN,ONSHORE SCENARIO,NO
562,Follow-up administrative tasks,"Provide account statements, investment reports...",Professionals,Pro-actively,EXISTING CLIENTS,1.0,NaN,NaN,NaN,ONSHORE SCENARIO,NO


In [21]:
all_df['OUTCOME'].value_counts()

YES                                                                  319
NO                                                                   229
?                                                                      8
NO without a fiduciary company (exceptional cases to be assessed)      4
YES with a fiduciary company and subject to annex 1                    4
Name: OUTCOME, dtype: int64